In [9]:
import numpy as np
from scipy.sparse import diags, csr_matrix
from scipy.sparse.linalg import norm
from scipy.optimize import minimize
from tensorflow.keras.datasets import mnist

# Load and preprocess MNIST data (using only the first 1000 samples)
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train[:10].reshape(-1, 28 * 28) / 255.0
y_train = y_train[:10]
class_assignment = np.zeros(y_train.shape[0])
class_assignment[(y_train == 4) | (y_train == 9)] = 1

# Construct the graph signal matrix S based on class assignment
S = class_assignment

# Reshape the images to 1D vectors
x_train_1d = x_train.reshape(-1, 28 * 28)

# Construct the adjacency matrix (1D chain graph) in sparse format
N = x_train_1d.shape[0]  # Number of nodes
adjacency = diags([1], [1], shape=(N, N)) + diags([1], [-1], shape=(N, N))
adjacency = adjacency.tocsr()

# Calculate the inverse square root of the sum of adjacency matrix rows
inv_sqrt_sum_rows = 1 / np.sqrt(adjacency.sum(axis=1)).A.flatten()

# Create the diagonal matrix D_inv_sqrt with the calculated values
D_inv_sqrt = diags([inv_sqrt_sum_rows], [0], shape=(N, N))

# Normalize adjacency matrix
Anorm = D_inv_sqrt @ adjacency @ D_inv_sqrt

# Define your optimization objective and gradient
def cost_function(S, alpha, M1, M2):
    CS_known = diagonal_matrix_C @ S
    CS_algorithm = C @ S
    diff_CS = CS_known - CS_algorithm
    J = 0.5 * (np.linalg.norm((S - (M1 + 2 * alpha * M2) @ S) @ Anorm)**2 +
               alpha * np.linalg.norm(diff_CS)**2)
    return J

def gradient(S, alpha, M1, M2):
    CS_known = diagonal_matrix_C @ S
    CS_algorithm = C @ S
    gradient_J = (M1 + 2 * alpha * M2) @ S - CS_known + CS_algorithm
    return gradient_J

# Define optimization function for scipy.optimize
def optimization_function(S):
    return cost_function(S, alpha, M1, M2), gradient(S, alpha, M1, M2)

# Set initial hyperparameters
alpha = 0.1
epsilon = 1e-6
max_iterations = 10

# Define C matrix based on class assignment
C = np.diag(class_assignment)

# Define diagonal_matrix_C as a global variable
diagonal_matrix_C = np.diag(S)

# Precompute matrices M1 and M2
M1 = (np.eye(S.shape[0]) - Anorm).T @ (np.eye(S.shape[0]) - Anorm)
M2 = C.T @ C

# Perform optimization with iterative updates
for iteration in range(10):  # Example: 10 iterations, you can adjust as needed
    result = minimize(optimization_function, S, method='L-BFGS-B', jac=True,
                      options={'maxiter': max_iterations, 'gtol': epsilon})
    
    # Extract the optimized signal matrix S and other relevant results
    optimized_S = result.x
    alpha *= 0.9  # Example: Reduce alpha by a factor, you can adjust
    epsilon *= 0.9  # Example: Reduce epsilon by a factor, you can adjust
    
    print(f"Iteration {iteration + 1}:")
    print("Optimized Signal Matrix S:")
    print(optimized_S)
    print(f"Updated alpha: {alpha}")
    print(f"Updated epsilon: {epsilon}")


Iteration 1:
Optimized Signal Matrix S:
[0.14740794 0.16612309 0.08552074 0.06649895 0.09951328 0.15471225
 0.16837304 0.15186142 0.12800852 0.08012764]
Updated alpha: 0.09000000000000001
Updated epsilon: 9e-07
Iteration 2:
Optimized Signal Matrix S:
[0.15619566 0.17866287 0.09931837 0.08036539 0.11183745 0.16529783
 0.17754337 0.15992259 0.13534082 0.08510549]
Updated alpha: 0.08100000000000002
Updated epsilon: 8.1e-07
Iteration 3:
Optimized Signal Matrix S:
[0.1645822  0.19105184 0.11372522 0.09498677 0.12453977 0.17568431
 0.18640788 0.1678776  0.14269576 0.09011889]
Updated alpha: 0.07290000000000002
Updated epsilon: 7.29e-07
Iteration 4:
Optimized Signal Matrix S:
[0.17246938 0.20313198 0.12850435 0.11010255 0.13742324 0.18574515
 0.1948675  0.1756298  0.15000455 0.09513778]
Updated alpha: 0.06561000000000002
Updated epsilon: 6.561000000000001e-07
Iteration 5:
Optimized Signal Matrix S:
[0.17978472 0.21476211 0.14340828 0.125437   0.15028801 0.19537299
 0.20284227 0.18308927 0.157